### Homework 8 (Sun Yi)

#### Load data

In [1]:
data_location = 'sqlite:///../../data/data.db'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

import pickle

In [3]:
def vol_ohlc(df, lookback=10):
    o = df.open
    h = df.high
    l = df.low
    c = df.close
    
    k = 0.34 / (1.34 + (lookback+1)/(lookback-1))
    cc = np.log(c/c.shift(1))
    ho = np.log(h/o)
    lo = np.log(l/o)
    co = np.log(c/o)
    oc = np.log(o/c.shift(1))
    oc_sq = oc**2
    cc_sq = cc**2
    rs = ho*(ho-co)+lo*(lo-co)
    close_vol = cc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    open_vol = oc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    window_rs = rs.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    result = (open_vol + k * close_vol + (1-k) * window_rs).apply(np.sqrt) * np.sqrt(252)
    result[:lookback-1] = np.nan
    
    return result

In [4]:
def plot_learning_curve(
    estimator,
    title,
    X,
    y,
    axes=None,
    ylim=None,
    cv=None,
    n_jobs=None,
    train_sizes=np.linspace(0.1, 1.0, 5),
    scoring=None
):
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator,
        X,
        y,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True,
        scoring=scoring,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    fit_time_argsort = fit_times_mean.argsort()
    fit_time_sorted = fit_times_mean[fit_time_argsort]
    test_scores_mean_sorted = test_scores_mean[fit_time_argsort]
    test_scores_std_sorted = test_scores_std[fit_time_argsort]
    axes[2].grid()
    axes[2].plot(fit_time_sorted, test_scores_mean_sorted, "o-")
    axes[2].fill_between(
        fit_time_sorted,
        test_scores_mean_sorted - test_scores_std_sorted,
        test_scores_mean_sorted + test_scores_std_sorted,
        alpha=0.1,
    )
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [149]:
ohlc = pd.read_sql('SELECT * FROM ohlc', data_location)
ohlc.head()

,ts,open,high,low,close,volume,volumeUSD,token,chain
0,2021-11-01 00:00:00,61421.37,61669.14,61239.60,61343.68,256.433869,1.575751e+07,BTC,BTC
1,2021-11-01 01:00:00,61346.17,61709.82,61171.22,61610.93,332.481185,2.044558e+07,BTC,BTC
2,2021-11-01 02:00:00,61610.94,61779.87,61299.89,61333.17,314.250720,1.935390e+07,BTC,BTC
3,2021-11-01 03:00:00,61333.17,61457.28,60050.00,60589.06,1059.931358,6.414625e+07,BTC,BTC
4,2021-11-01 04:00:00,60590.23,60655.00,59752.92,59971.89,621.419878,3.744744e+07,BTC,BTC


In [150]:
ohlc.describe()

,open,high,low,close,volume,volumeUSD
count,11627.000000,11627.000000,11627.000000,11627.000000,1.162700e+04,1.162700e+04
mean,5708.198992,5737.512791,5676.641523,5706.967946,7.782893e+05,9.847622e+06
std,16518.161143,16599.532113,16430.972527,16514.731530,2.057505e+06,1.769057e+07
min,0.999900,1.000000,0.995100,0.999900,6.713000e+00,1.960784e+03
25%,4.555900,4.611000,4.496050,4.554350,2.565695e+03,9.664755e+05
50%,92.590000,93.710000,91.000000,92.600000,4.624230e+04,3.420994e+06
75%,307.924500,309.700000,305.501000,307.796500,1.768436e+05,1.068309e+07
max,68638.470000,69000.000000,68456.500000,68639.630000,3.978895e+07,3.988035e+08


#### Format data and generate features

In [151]:
tokens = ohlc.token.unique()

In [395]:
def df_merge(left, right):
    return pd.merge(left, right, on='ts', how='inner')

X = reduce(df_merge, [
    (lambda df: 
    (
        df
        .assign(
            vol=vol_ohlc(df).fillna(0),
            ret=df.close.pct_change(),
            hl_diff=df.high / df.low - 1,              # high - low
            volume_chg=df.volumeUSD.pct_change(),     # % volume change from last hour
            volume_momt=df.volumeUSD.pct_change().apply(np.sign).rolling(24).sum()/24,  # number of improving volume hours in past day
            ma_6h=df.close/df.close.rolling(6).mean()-1,
            ma_24h=df.close/df.close.rolling(24).mean()-1,
            ret_daily=df.close/df.close.shift(24)-1,
            ret_12h=df.close/df.close.shift(12)-1,
            ret_6h=df.close/df.close.shift(6)-1,
            ret_3h=df.close/df.close.shift(3)-1,
            drawdown= df.close / df.high - 1,          
            bounce=df.close / df.low - 1
        )[['ts', 'vol', 'ret', 
           'hl_diff', 
           'volume_chg', 
           # 'volume_momt',
           # 'ma_6h',
           # 'ma_24h',
           # 'ret_daily', 
           # 'ret_12h', 
           # 'ret_6h', 
           # 'ret_3h', 
           'drawdown', 
           'bounce',
          ]]
        .rename(columns={
            col: f'{col}_{token}' for col in ['ts', 'vol', 'ret', 
                                              'hl_diff', 
                                              'volume_chg',
                                              # 'volume_momt',
                                              # 'ma_6h',
                                              # 'ma_24h',
                                              # 'ret_daily', 
                                              # 'ret_12h', 
                                              # 'ret_6h', 
                                              # 'ret_3h', 
                                              'drawdown', 
                                              'bounce',
                                             ] if col != 'ts'
        })
    ))(ohlc[ohlc.token == token])
    for token in tokens
]).set_index('ts')

X['weekday'] = (pd.DatetimeIndex(X.index).day_of_week < 5).astype(float)

y = X.ret_SOL.shift(-1)[:-1]
X = X[:-1]

In [396]:
X.shape

(1056, 67)

In [397]:
X

,vol_BTC,ret_BTC,hl_diff_BTC,volume_chg_BTC,drawdown_BTC,bounce_BTC,vol_ETH,ret_ETH,hl_diff_ETH,volume_chg_ETH,...,volume_chg_AAVE,drawdown_AAVE,bounce_AAVE,vol_COMP,ret_COMP,hl_diff_COMP,volume_chg_COMP,drawdown_COMP,bounce_COMP,weekday
ts,,,,,,,,,,,,,,,,,,,,,
2021-11-01 00:00:00,0.000000,NaN,0.007014,NaN,-0.005278,0.001700,0.000000,NaN,0.007397,NaN,...,NaN,-0.002610,0.004567,0.000000,NaN,0.022303,NaN,-0.000577,0.021713,1.0
2021-11-01 01:00:00,0.000000,0.004357,0.008805,0.297514,-0.001603,0.007188,0.000000,0.006874,0.010912,0.866676,...,4.209224,-0.004486,0.012734,0.000000,-0.002281,0.012354,-0.407882,-0.005469,0.006818,1.0
2021-11-01 02:00:00,0.000000,-0.004508,0.007830,-0.053394,-0.007231,0.000543,0.000000,-0.005322,0.007534,-0.405713,...,-0.695077,-0.010831,0.000000,0.000000,-0.006020,0.021778,1.038275,-0.020002,0.001341,1.0
2021-11-01 03:00:00,0.000000,-0.012132,0.023435,2.314383,-0.014127,0.008977,0.000000,-0.013126,0.027325,1.713614,...,2.169505,-0.016597,0.011419,0.000000,-0.022273,0.035888,0.269205,-0.025621,0.009348,1.0
2021-11-01 04:00:00,0.000000,-0.010186,0.015097,-0.416218,-0.011262,0.003665,0.000000,-0.010679,0.016003,-0.373503,...,-0.309997,-0.019598,0.001972,0.000000,-0.024002,0.029444,-0.346529,-0.027563,0.001069,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-14 19:00:00,0.136406,0.000402,0.010925,-0.180185,-0.006681,0.004171,0.159860,-0.002922,0.015699,-0.114507,...,0.399707,-0.012361,0.010853,0.286230,-0.011428,0.032641,-0.675907,-0.020781,0.011181,1.0
2021-12-14 20:00:00,0.136358,0.004810,0.009388,0.096814,-0.004445,0.004902,0.158369,0.005961,0.009721,-0.177999,...,0.019698,-0.005080,0.011311,0.284268,0.006813,0.014268,-0.288347,-0.005461,0.008729,1.0
2021-12-14 21:00:00,0.142237,0.019797,0.022073,1.110197,-0.001773,0.020261,0.170096,0.016737,0.024448,1.213437,...,1.023374,-0.007903,0.021931,0.281497,0.017473,0.021116,2.932668,-0.001796,0.019282,1.0


#### Set up transformers

In [158]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR

from sklearn.model_selection import cross_validate
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer

from sklearn.model_selection import learning_curve

In [159]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]


In [160]:
def evaluate_model(model, X, y, test_size=0.2):
    cv = TimeSeriesSplit(n_splits=int(y.shape[0] * test_size), test_size=1)
    scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)
    
    return np.mean(cross_validate(model, X, y, cv=cv, scoring=scorer, n_jobs=-1)['test_score'])
    
#     scores = []
#     for train_idx, test_idx in cv.split(X):
#         X_train, X_test, y_train, y_test = X.iloc[train_idx], X.iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]
#         model.fit(X_train, y_train)
#         score = mean_squared_error(y_test, model.predict(X_test), squared=False)
#         scores.append(score)
        
#     return np.mean(scores)

#### Test out different models

In [398]:
pipeline = Pipeline([
    ('impute', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0.)),
    ('pca', PCA(n_components=10)),
    ('model', Ridge(alpha=0.2))
])

evaluate_model(pipeline, X, y)

-0.008496577600466031

In [399]:
pipeline = Pipeline([
    ('impute', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0.)),
    ('pca', PCA(n_components=10)),
    ('model', Ridge(alpha=0.1))
])

evaluate_model(pipeline, X, y)

-0.008496576868127848

In [401]:
pipeline = Pipeline([
    ('impute', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0.)),
    ('pca', PCA(n_components=10)),
    ('model', Lasso(alpha=0.1))
])

evaluate_model(pipeline, X, y)

-0.008570816681474391

In [402]:
pipeline = Pipeline([
    ('impute', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0.)),
    # ('scale', StandardScaler()),
    ('model', ElasticNet(alpha=0.1, l1_ratio=0.5))
])

evaluate_model(pipeline, X, y)

-0.008570816681474391

In [403]:
pipeline = Pipeline([
    ('impute', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0.)),
    ('scale', StandardScaler()),
    ('model', SVR(epsilon=0.2, kernel='rbf'))
])

evaluate_model(pipeline, X, y)

-0.008546201345938906

#### Tune hyperparameters

In [408]:
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('impute', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0.)),
    # ('scale', StandardScaler()),
    ('pca', PCA()),
    ('model', Ridge())
])

test_size = 0.2
cv = TimeSeriesSplit(n_splits=int(y.shape[0] * test_size), test_size=1)
scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

search = GridSearchCV(pipeline, {
    'pca__n_components': range(1,20,1),
    'model__alpha': np.arange(0.01,0.2,0.02)
}, scoring=scorer, refit=True, cv=cv, n_jobs=-1)
search.fit(X, y)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=211, test_size=1),
             estimator=Pipeline(steps=[('impute',
                                        SimpleImputer(fill_value=0.0,
                                                      strategy='constant')),
                                       ('pca', PCA()), ('model', Ridge())]),
             n_jobs=-1,
             param_grid={'model__alpha': array([0.01, 0.03, 0.05, 0.07, 0.09, 0.11, 0.13, 0.15, 0.17, 0.19]),
                         'pca__n_components': range(1, 20)},
             scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))

In [409]:
search.best_params_

{'model__alpha': 0.01, 'pca__n_components': 11}

In [410]:
best_model = search.best_estimator_
evaluate_model(best_model, X, y)

-0.008469964312173502